In [ ]:
import os
import numpy as np
import cv2

from matplotlib import pyplot as plt
import matplotlib 

from time import time
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

# Carga conjunto de datos

Se proporciona la carpeta, a través de la variable folder, donde cada subcarpeta se corresponde con una clase. Cada clase contiene muestras en forma de imágenes jpg, todas del mismo tamaño

In [ ]:
#MODIFICAR INDICANDO RUTA EN TU EQUIPO TRAS DESCAGAR DATOS DEL CAMPUS
folder = "C:/Users/otsed/Desktop/Docencia/VC/DatabaseGender59x65" #Portátil

# Contador de número de clases del conjunto
nclasses = 0
# Contador de muestras por clase
nperclass = []
# Etiqueta de cada clase (nombre de la subcarpeta)
classlabels = []
# Inicializa estructuras de datos y sus correpondientes etiquetas
X = []
Y = []

# Valor de resolución por defecto de matplotlib
dpi = matplotlib.rcParams['figure.dpi']

# Asume que en la ruta indicada hay una subcarpeta por clase
for class_name in os.listdir(folder):
    # Cada subcarpeta implica una clase más
    nclasses += 1
    # Inicialmente esta clase no tiene muestras
    nsamples = 0

    # Compone la ruta
    class_folder = os.path.join(folder, class_name)
    for file_name in os.listdir(class_folder):
        # Asume imágenes en formato jpg
        if file_name.endswith('.jpg'):
            # Lee la imagen
            image = cv2.imread (os.path.join(class_folder, file_name),)
            
            # Lugar para aplicar un RoI 
            # Ejemplo para la mitad superior
            # image = image[0:int(height/2),:]
            
            # Extrae tamaños
            height, width, depth = image.shape
            # Convierte  grises
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # Añade a X, previa conversión en vector
            X.append(gray.reshape(height*width))
            # Añade etiqueta numérica de la muestra
            Y.append(nclasses-1)
            # Muestra primera imagen de la clase
            if nsamples == 0:
                figsize = width / float(dpi), height / float(dpi)

                plt.figure(figsize=figsize)
                plt.imshow(gray, cmap='gray', vmin=0, vmax=255)
                plt.show()
  
            #Incrementa el número de muestras
            nsamples += 1
    nperclass.append(nsamples)
    classlabels.append(class_name)
    
#Convierte a numpy array X e Y
X = np.array(X,dtype='float32')
Y = np.array(Y,dtype='float64')

# Muestra datos del conjunto leído
# Depuración
#print(X.shape)
#print(Y.shape)
# Obtiene número de muestras y características
n_samples , n_features = X.shape
# Obtiene nombres de las clases
class_names = np.array(classlabels)
n_classes = class_names.shape[0]

print("Dataset info:")
print("# samples: %d" % n_samples)
print("# features: %d" % n_features)
print("# classes: %d" % n_classes)
print("classes %s" % classlabels)
print("samples per class %s" % str(nperclass)[1:-1] )

# Diseña conjunto experimental 70/30

Divide los datos en conjunto de entrenamiento y test

In [ ]:
# Split into a train (70%) and test (30%)
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.3, random_state=42)

print("# samples in training set %d" % X_train.shape[0])
print("# samples in test set %d" % X_test.shape[0])

# Píxeles como descriptores y KNN con distancia euclídea para clasificación

In [ ]:
#KNN utilizando píxeles 
print("KNN classifier based on pixels\nTraining...")
t0 = time()
# k = 5 
model_px = KNeighborsClassifier(n_neighbors = 5) 
  
# fdtraining of model 
model_px.fit(X_train, y_train) 
print("done in %0.3fs" % (time() - t0))

print("Predicting...")
t0 = time()
y_pred=model_px.predict(X_test)

print("done in %0.3fs" % (time() - t0))

print("Classification results:")
print(classification_report(y_test, y_pred, target_names=class_names))
print("Precision:  %0.3f, Recall:  %0.3f" % ( precision_score(y_test, y_pred),recall_score(y_test, y_pred) )) 
print("Confusion matrix")
print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))

# Cálculo de PCA (150 componentes) 

In [ ]:
# Compute PCA on the training subset for n_comp components
n_comp = 150
print("PCA computation using %d components from %d faces..."
      % (n_comp, X_train.shape[0]))
t0 = time()
pca_n = PCA(n_components=n_comp, svd_solver='randomized',
          whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

eigenfaces_n = pca_n.components_.reshape((n_comp, height, width))

print("Projecting training and test on the eigenfaces orthonormal basis...")
t0 = time()
X_train_pca_n = pca_n.transform(X_train)
X_test_pca_n = pca_n.transform(X_test)
print("done in %0.3fs" % (time() - t0))

# Cara media
plt.imshow(pca_n.mean_.reshape(height,width),            cmap=plt.cm.bone)

fig = plt.figure(figsize=(16, 6))
for i in range(30):
    ax = fig.add_subplot(3, 10, i + 1, xticks=[], yticks=[])
    ax.imshow(pca_n.components_[i].reshape(height,width),
              cmap=plt.cm.bone)

# PCA (150 componentes) como descriptores y KNN con distancia euclídea para clasificación

In [ ]:
#KNN for k neighbors
k = 5
print("KNN (k=%d) classifier based on %d PCA components..." % (k, n_comp) ) 
t0 = time()
model_pca_n = KNeighborsClassifier(n_neighbors = k) 
  
# fdtraining of model 
model_pca_n.fit(X_train_pca_n, y_train) 
print("done in %0.3fs" % (time() - t0))

print("Predicting...")
t0 = time()
y_pred=model_pca_n.predict(X_test_pca_n)

print("done in %0.3fs" % (time() - t0))

print("\nMetrics")
print(classification_report(y_test, y_pred, target_names=class_names))
print("Precision:  %0.3f, Recall:  %0.3f" % ( precision_score(y_test, y_pred),recall_score(y_test, y_pred) )) 
print("Confusion matrix")
print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))

# PCA (150 componentes) como descriptores y SVM para clasificación

In [ ]:
# Train a SVM classifier
print("SVM training...")
t0 = time()
parameters = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],}
# Grid search across parameter range
clf_pca_n = GridSearchCV(
    SVC(kernel='rbf', class_weight='balanced'), parameters, cv=5
)
clf_pca_n = clf_pca_n.fit(X_train_pca_n, y_train)
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf_pca_n.best_estimator_)


print("Predicting")
t0 = time()
y_pred = clf_pca_n.predict(X_test_pca_n)
print("done in %0.3fs" % (time() - t0))

print("\nMetrics")
print(classification_report(y_test, y_pred, target_names=class_names))
print("Precision:  %0.3f, Recall:  %0.3f" % ( precision_score(y_test, y_pred),recall_score(y_test, y_pred) )) 
print("Confusion matrix")
print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))

# Cálculo de PCA (95% de la varianza)

In [ ]:
# ompute PCA on the training subset for n_comp components
print("PCA computation for .95 of the variance from %d faces..."
      % (X_train.shape[0]))
t0 = time()
pca_95 = PCA(.85).fit(X_train)
print("done in %0.3fs" % (time() - t0))
print("%d components selected" % (pca_95.n_components_ ))

eigenfaces_95 = pca_95.components_.reshape((pca_95.n_components_ , height, width))

print("Projecting training and test on the eigenfaces orthonormal basis")
t0 = time()
X_train_pca_95 = pca_95.transform(X_train)
X_test_pca_95 = pca_95.transform(X_test)
print("done in %0.3fs" % (time() - t0))

# PCA (95% varianza) como descriptores y KNN con distancia euclídea para clasificación

In [ ]:
#KNN for k neighbors
k = 5
print("KNN (k=%d) classifier based on %d PCA components..." % (k, pca_95.n_components_) ) 
t0 = time()
model_pca_95 = KNeighborsClassifier(n_neighbors = k) 
  
# fdtraining of model 
model_pca_95.fit(X_train_pca_95, y_train) 
print("done in %0.3fs" % (time() - t0))

print("Predicting...")
t0 = time()
y_pred=model_pca_95.predict(X_test_pca_95)

print("done in %0.3fs" % (time() - t0))

print("\nMetrics")
print(classification_report(y_test, y_pred, target_names=class_names))
print("Precision:  %0.3f, Recall:  %0.3f" % ( precision_score(y_test, y_pred),recall_score(y_test, y_pred) )) 
print("Confusion matrix")
print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))

# PCA (95% varianza) como descriptores y SVM para clasificación 

In [ ]:
# Train a SVM classifier
# Data normalization according to training data before training
print("Normalization...")
scaler = MinMaxScaler()
train_X = scaler.fit_transform(X_train_pca_95)
test_X = scaler.transform(X_test_pca_95)

print("SVM training...")
t0 = time()
parameters = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],}
# Grid serach across parameter range
clf_pca_95 = GridSearchCV(
    SVC(kernel='rbf', class_weight='balanced'), parameters, cv=5
)
clf_pca_95 = clf_pca_95.fit(train_X, y_train)
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf_pca_95.best_estimator_)


print("Predicting")
t0 = time()
y_pred = clf_pca_95.predict(test_X)
print("done in %0.3fs" % (time() - t0))

print("\nMetrics")
print(classification_report(y_test, y_pred, target_names=class_names))
print("Precision:  %0.3f, Recall:  %0.3f" % ( precision_score(y_test, y_pred),recall_score(y_test, y_pred) )) 
print("Confusion matrix")
print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))